In [1]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
import time
import torch.optim as optim
import torch.nn as nn
import numpy as np
import os
import matplotlib.pyplot as plt
from pathlib import Path
import torch
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader,Dataset
from tqdm.notebook import tqdm # Progession bar
from PIL import Image
import torchvision.models as models
from torchvision.models import ResNet50_Weights
from sklearn.metrics import normalized_mutual_info_score
from sklearn.cluster import AgglomerativeClustering

In [2]:

src_path = os.path.abspath('../src')
if src_path not in sys.path:
    sys.path.append(src_path)

data_dir = os.path.join('..', 'data')
metadata_dir = os.path.join(data_dir, 'metadata')
test_csv = os.path.join(metadata_dir, 'test_metadata.csv')
train_csv = os.path.join(metadata_dir, 'train_metadata.csv')
val_csv = os.path.join(metadata_dir, 'val_metadata.csv')

images_dir = os.path.join(data_dir,'images')
processed_dir = os.path.join(data_dir,'processed')
model_dir = os.path.join('..','models','jule')

In [5]:
def make_data_loaders_unsupervised(train_csv, val_csv, test_csv,processed_dir, images_dir, batch_size, image_size):
    from torch.utils.data import DataLoader, Dataset
    from PIL import Image

    class ImageDataset(Dataset):
        def __init__(self, csv_file, root_dir, transform=None):
            self.data = pd.read_csv(csv_file)
            self.root_dir = root_dir
            self.transform = transform

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
            image = Image.open(img_name).convert('RGB')

            # Obtener las etiquetas (asumiendo que empiezan desde el segundo índice)
            labels = self.data.iloc[idx, 1:].values.astype(np.float32)

            if self.transform:
                image = self.transform(image)

            return image, labels

    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_dataset = ImageDataset(train_csv, processed_dir, transform)
    val_dataset = ImageDataset(val_csv, images_dir, transform)
    test_dataset = ImageDataset(test_csv, images_dir, transform)

    dataloaders = {
        'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
        'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=False),
        'test': DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    }
    return dataloaders


In [9]:
dataloaders = make_data_loaders_unsupervised(train_csv,val_csv,test_csv,processed_dir,images_dir,33,224)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [7]:
def perform_clustering(embeddings, n_clusters=14):
    clustering_model = AgglomerativeClustering(n_clusters=n_clusters, affinity='euclidean', linkage='ward')
    cluster_labels = clustering_model.fit_predict(embeddings)
    return cluster_labels

class ResNet50Encoder(nn.Module):
    def __init__(self, embedding_dim=128):
        super(ResNet50Encoder, self).__init__()
        resnet = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        self.features = nn.Sequential(*list(resnet.children())[:-1]) 
        self.fc = nn.Linear(resnet.fc.in_features, embedding_dim)  
        self.normalize = nn.functional.normalize  

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return self.normalize(x, p=2, dim=1)

In [8]:
def load_model_and_generate_embeddings(model_path, dataloader, model, device):
    # Load model weights
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    all_embeddings = []
    all_labels = []
    
    with torch.no_grad():
        for imgs, labels in dataloader:
            imgs = imgs.to(device)
            embeddings = model(imgs)
            all_embeddings.append(embeddings.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
    
    all_embeddings = np.vstack(all_embeddings)
    all_labels = np.hstack(all_labels)
    return all_embeddings, all_labels


def calculate_nmi(model_path, dataloader, model, device, n_clusters=14):
    # Step 1: Generate embeddings
    embeddings, true_labels = load_model_and_generate_embeddings(model_path, dataloader, model, device)
    
    # Step 2: Perform clustering on embeddings
    cluster_labels = perform_clustering(embeddings, n_clusters=n_clusters)
    
    # Step 3: Compute NMI
    nmi_score = normalized_mutual_info_score(true_labels, cluster_labels)
    print(f"NMI Score: {nmi_score:.4f}")
    return nmi_score






NameError: name 'dataloaders' is not defined

In [11]:
# Cargar el modelo y calcular NMI en el conjunto de test
model = ResNet50Encoder().to(device)

# Cargar el modelo entrenado desde el archivo .pth
model.load_state_dict(torch.load("model_epoch_4.pth"))
# Establecer el modelo en modo evaluación (importante para evitar dropout y batchnorm)
model.eval()
# model_path = "model_jule_epoch_4.pth"
# test_dataloader = dataloaders['test']
# nmi_score = calculate_nmi(model_path, test_dataloader, model, device, n_clusters=18)

C:\Users\MEDHYCON\AppData\Local\Temp\ipykernel_7924\3442251958.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("model_epoch_4.pth"))


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.